In [1]:
# установим библиотеку для анализа тональности русского языка (https://pypi.org/project/dostoevsky/)
# install the library for sentiment analysis of the Russian language
#!pip install dostoevsky

In [2]:
# загрузим предобученную модель
# download pretrained model
#!python -m dostoevsky download fasttext-social-network-model

In [3]:
# data preprocessing
import pandas as pd
import re

# Sentiment analysis
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

## 1. Загрузка и предобработка данных. 
## 1. Load and preprocessing data

In [4]:
file = open('/content/test_margaryta.txt','r',encoding='utf-8')
text = file.readlines()

In [5]:
print(len(text))
print(type(text))

8
<class 'list'>


In [6]:
text[0]

"{'url': 'news/2020/05/11/prodazhi-avtovaza-v-aprele-snizilis-v-tri-raza-po-sravneniyu-s-proshlym-godom', 'title': 'Продажи «АвтоВАЗа» в апреле снизились в три раза по сравнению с прошлым годом', 'article': ['«АвтоВАЗ» в\\xa0апреле продал в\\xa0общей сложности 10055\\xa0автомобилей, сообщила пресс-служба автоконцерна. Самой популярной была Granta с\\xa0результатом 3058 автомобилей, на\\xa0втором месте Vesta\\xa0— 2913 проданных автомобилей, на\\xa0третьем\\xa0— Largus\\xa0— 1286 автомобилей.', 'По\\xa0сравнению с\\xa0апрелем 2019 года, сообщает ТАСС, продажи «АвтоВАЗа» упали более чем в\\xa03 раза. В\\xa0прошлом году автоконцерн продал в\\xa0апреле\\xa032,3 тысячи автомобилей.']},\n"

In [7]:
# Переходим от формата str к dict

d = [eval(string[:-2]) for string in text]

In [8]:
#  Save to dataframe
data = pd.DataFrame.from_dict(d)
data.head(3)

,url,title,article
0,news/2020/05/11/prodazhi-avtovaza-v-aprele-sni...,Продажи «АвтоВАЗа» в апреле снизились в три ра...,[«АвтоВАЗ» в апреле продал в общей сложности 1...
1,news/2020/05/11/vms-irana-vo-vremya-ucheniy-sl...,ВМС Ирана во время учений случайно выпустили р...,"[Противокорабельная ракета, запущенная во врем..."
2,news/2020/05/11/rossiya-vyshla-na-tretie-mesto...,Россия вышла на третье место в мире по уровню ...,[В России за сутки коронавирусную инфекцию выя...


In [9]:
data.article[0]

['«АвтоВАЗ» в\xa0апреле продал в\xa0общей сложности 10055\xa0автомобилей, сообщила пресс-служба автоконцерна. Самой популярной была Granta с\xa0результатом 3058 автомобилей, на\xa0втором месте Vesta\xa0— 2913 проданных автомобилей, на\xa0третьем\xa0— Largus\xa0— 1286 автомобилей.',
 'По\xa0сравнению с\xa0апрелем 2019 года, сообщает ТАСС, продажи «АвтоВАЗа» упали более чем в\xa03 раза. В\xa0прошлом году автоконцерн продал в\xa0апреле\xa032,3 тысячи автомобилей.']

In [10]:
# функция по очистке html тегов
# function for cleaning html tags

def remove_html_tags(str_line):
    tmp = str_line.replace(u'\xa0', u' ')
    return re.sub('<[^<]+?>', '', tmp)

In [11]:
# почистим данные от html тегов и объединим в 1 строку
# update our data by cleaning and joining article texts

data.article = list(map(lambda x: remove_html_tags(str.join(',\n', x)), data.article))

In [12]:
data.article[2]

'В России за сутки коронавирусную инфекцию выявили у 11656 человек, сообщает оперативный штаб по борьбе с распространением COVID-19.,\nОбщее число зарегистрированных случаев в России достигло 221344. По этому показателю Россия вышла на третье место, обойдя Италию и Великобританию.,\nБольше всего новых случаев коронавируса выявили в Москве (+6169 человек), Подмосковье (+1079), Санкт-Петербурге (+307), Нижегородской области (+291), Смоленской области (+164) и Дагестане (+119). В Москве число выявленных случаев заболевания достигло 115909.,\nЗа сутки зарегистрировали 94 смерти от коронавируса. Всего с начала эпидемии, по официальным данным, умерли 2009 человек, у которых диагностировали заболевание.,\nС начала распространения COVID-19 в России выздоровели 39801 человек (+5495 за сутки). В Москве за сутки выздоровели 4032 человека, за все время — 17822 человека.,\nЗа прошедшие сутки, сообщает «Интерфакс» со ссылкой на Роспотребнадзор, в России было выполнено около 188 тысяч тестов на корон

В тексте статей остался символ переноса строки'\n'. Давайте уберем его

There are '\n' in the text. Let's remove it.

In [13]:
data.article = data.article.str.replace(u'\n', u' ')

In [14]:
data.article[2]

'В России за сутки коронавирусную инфекцию выявили у 11656 человек, сообщает оперативный штаб по борьбе с распространением COVID-19., Общее число зарегистрированных случаев в России достигло 221344. По этому показателю Россия вышла на третье место, обойдя Италию и Великобританию., Больше всего новых случаев коронавируса выявили в Москве (+6169 человек), Подмосковье (+1079), Санкт-Петербурге (+307), Нижегородской области (+291), Смоленской области (+164) и Дагестане (+119). В Москве число выявленных случаев заболевания достигло 115909., За сутки зарегистрировали 94 смерти от коронавируса. Всего с начала эпидемии, по официальным данным, умерли 2009 человек, у которых диагностировали заболевание., С начала распространения COVID-19 в России выздоровели 39801 человек (+5495 за сутки). В Москве за сутки выздоровели 4032 человека, за все время — 17822 человека., За прошедшие сутки, сообщает «Интерфакс» со ссылкой на Роспотребнадзор, в России было выполнено около 188 тысяч тестов на коронавиру

## 2. Анализ тональности
## 2. Sentiment analysis

In [15]:
tokenizer = RegexTokenizer()

In [16]:
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [17]:
results = model.predict(data.article, k=2) 

In [18]:
results

[{'neutral': 0.9284188151359558, 'negative': 0.11280541121959686},
 {'neutral': 0.7606606483459473, 'negative': 0.20182321965694427},
 {'neutral': 0.6513648629188538, 'negative': 0.23935934901237488},
 {'neutral': 0.6654205918312073, 'negative': 0.23371636867523193},
 {'neutral': 0.7371681928634644, 'negative': 0.24509501457214355},
 {'neutral': 0.7879412174224854, 'negative': 0.13661839067935944},
 {'neutral': 0.6370407938957214, 'negative': 0.20182321965694427},
 {'neutral': 0.6297846436500549, 'negative': 0.18714269995689392}]

У нас нет ни одной положительной оценки для статей, только нейтральная и негативная. Вероятнее всего подобный перекос в сторону "нейтральности" текстов связан с их относительно большим объемом.

Давайте определим эмоциональную оценку текста для заголовков (они меньше размром и  вних стараются отразить суть статьи).


---

We do not have a single positive rating for articles, only neutral and negative. Most likely, the appearance of a bias on the side of the "neutrality" of the texts, associated with their relatively large volume.

Let's define the emotional value of the text for the headlines (they are smaller in size and they try to reflect the essence of the article).

In [19]:
results_title = model.predict(data.title, k=2) 

In [20]:
results_title

[{'neutral': 0.812877357006073, 'negative': 0.275139719247818},
 {'neutral': 0.4921981394290924, 'negative': 0.1968362182378769},
 {'neutral': 0.6001983880996704, 'negative': 0.2814156115055084},
 {'neutral': 0.9399233460426331, 'skip': 0.06755668669939041},
 {'neutral': 0.7981967926025391, 'negative': 0.11921291798353195},
 {'neutral': 0.275139719247818, 'negative': 0.15611489117145538},
 {'neutral': 0.8354935646057129, 'negative': 0.33459946513175964},
 {'neutral': 0.3276783227920532, 'positive': 0.07370626926422119}]

In [21]:
for message, sentiment in zip(data.title, results_title):
  print(message, '->', sentiment, '\n')

Продажи «АвтоВАЗа» в апреле снизились в три раза по сравнению с прошлым годом -> {'neutral': 0.812877357006073, 'negative': 0.275139719247818} 

ВМС Ирана во время учений случайно выпустили ракету по своему кораблю. Погибли 19 человек -> {'neutral': 0.4921981394290924, 'negative': 0.1968362182378769} 

Россия вышла на третье место в мире по уровню распространения COVID-19. С момента начала эпидемии умерли более 2000 человек -> {'neutral': 0.6001983880996704, 'negative': 0.2814156115055084} 

В МВД назвали регионы, лидирующие по числу нарушений режима изоляции -> {'neutral': 0.9399233460426331, 'skip': 0.06755668669939041} 

И.о. главы Россотрудничества в Чехии заподозрили в доставке рицина в Прагу -> {'neutral': 0.7981967926025391, 'negative': 0.11921291798353195} 

Смертность в Москве в апреле достигла пика за десятилетие -> {'neutral': 0.275139719247818, 'negative': 0.15611489117145538} 

Всех постарался придушить -> {'neutral': 0.8354935646057129, 'negative': 0.33459946513175964} 



* В 4-м заголовке появилась новоая оценка "skip", что означает то, что нейронная сеть не смогла разобрать эту фразу и скорее всего там отсутствует эмоциональный контекст.

* В последнем заголовке текст был оценен как "нейтральный" в 0.32 балла, и как "позитивный" в 0.07 баллов, хотя явно максимальная оценка должна быть у класса  "негативный" так как в заголовке фигурируют такие понятия как "пожар" и "погибли 9 человек". 

* Данная библиотека плохо работает с данным датасетом, необходимо попробовать другие методы и подходы.



---

* In the 4th heading, a new score "skip" appeared, which means that the neural network could not parse this phrase and most likely there is no emotional context there.

* In the last heading, the text was rated as "neutral" at 0.32 points, and as "positive" at 0.07 points, although the class "negative" should clearly have the maximum rating, since the heading includes such concepts as "fire" and "9 people died ".

* This library does not work well with this dataset, you need to try other methods and approaches.
